In [1]:
#pip install trino urllib3

In [2]:
import trino
import urllib3
import os

urllib3.disable_warnings()

## Enforcing Access - user groups

The following cell establishes two separate trino db connections:
 - Connection 1 - user `hukhan` - member of trino user group `team1`
 - Connection 2 - user `mikhail` - member of trino user group `team2`
 
Trino also allows access controls on a per-user basis.
However configuring access by user-group is best practice,
as it allows individual user permissions to be controlled at the group-membership level,
which keeps the trino access rules simple and low-churn

In [4]:
conn_team1 = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication('hukhan', 'osctrino987'),
    host='trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org',
    port=443,
    http_scheme='https',
    verify=False,
)
cur_team1 = conn_team1.cursor()

conn_team2 = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication('mikhail', 'osctrino987'),
    host='trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org',
    port=443,
    http_scheme='https',
    verify=False,
)
cur_team2 = conn_team2.cursor()

## Enforcing Access - visibility

The following queries show which schemas each user-group has access to

In [5]:
cur_team1.execute('show schemas in hive')
cur_team1.fetchall()

[['team1']]

In [6]:
cur_team2.execute('show schemas in hive')
cur_team2.fetchall()

[['team2']]

## Enforcing access - success

The following two queries will suceed,
because the users belong to user-groups that can read their respective schemas

In [7]:
cur_team1.execute('select * from hive.team1.cat')
cur_team1.fetchall()

[['word', 'length'],
 ['the', '3'],
 ['cat', '3'],
 ['in', '2'],
 ['the', '3'],
 ['hat', '3']]

In [8]:
cur_team2.execute('select * from hive.team2.cat')
cur_team2.fetchall()

[['word', 'length'],
 ['the', '3'],
 ['cat', '3'],
 ['in', '2'],
 ['the', '3'],
 ['hat', '3']]

## Enforcing access - failures

The following two queries will fail with a python exception,
Because the users do not have access to the schemas they are trying to read

In [11]:
cur_team1.execute('select * from hive.team2.cat')
cur_team1.fetchall()

# Expected failure:
# message="Access Denied: Cannot select from table hive.team2.cat"

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot select from table hive.team2.cat", query_id=20210831_180159_00018_72mua)

In [12]:
cur_team2.execute('select * from hive.team1.cat')
cur_team2.fetchall()

# Expected failure:
# message="Access Denied: Cannot select from table hive.team1.cat"

TrinoUserError: TrinoUserError(type=USER_ERROR, name=PERMISSION_DENIED, message="Access Denied: Cannot select from table hive.team1.cat", query_id=20210831_180208_00019_72mua)